# Доказательства

## Решаем задания

In [1]:
from z3 import *

### Задание

Найдите корень уравнения `x**3 + 3*x**2 + 4*x + 2 == 0` с помощью Z3. Сможете вы применить Z3 так, чтобы доказать, что найденный корень единственный, или опровергнуть это утверждение?

In [ ]:
x = Real("x")
solve(x**3 + 3*x**2 + 4*x + 2 == 0)  # = -1

claim = ForAll(
    x,
    Implies(x**3 + 3*x**2 + 4*x + 2 == 0, x == -1)
)  # для всех х верно следствие равенство истино -> когда х = -1

prove(claim)


## Задание на исчёрпывающее отсутствие контрпримеров

Докажите второе правило самостоятельно.
- отрицание дизъюнкции есть конъюнкция отрицаний `p | q == ~ (~p & ~q)`

In [ ]:
def prove2(f): # упрощенная версия функции prove() Z3
    s = Solver()
    s.add(Not(f))
    ch = s.check()
    if s.check() == unsat:
        print("доказано")
    else:
        print("опровергнуто")

p = Bool("p")
q = Bool("q")

prove(Or(p, q) == Not(And(Not(p), Not(q))))
prove2(Or(p, q) == Not(And(Not(p), Not(q))))

## 2.5. Задание на доказательства в арифметике

Докажите (или найдите контрпримеры) следующие теоремы для целых, рациональных, вещественных и битовых чисел:

- `x + y == y + x`

- `(x + y) + z == x + (y + z)`

- x может быть меньше, больше или равен 0;

- x в квадрате всегда положителен.


In [ ]:
# Решение

# целые
x = Int("x")
y = Int("y")
z = Int("z")

prove(x + y == y + x) # коммутативность
prove(((x + y) + z) == ((x + (y + z)))) # ассоциативность
prove(x + 0 == x) # существование "0"
prove(1 * x == x) # существование "1"
prove(Or(x > 0, x < 0, x == 0))
prove(x * (y + z) == x * y + x * z)
prove(x ** 2 >= 0)


# рациональные
x = Real("x")
y = Real("y")
z = Real("z")

prove(x + y == y + x) # коммутативность
prove(((x + y) + z) == ((x + (y + z)))) # ассоциативность
prove(x + 0 == x) # существование "0"
prove(1 * x == x) # существование "1"
prove(Or(x > 0, x < 0, x == 0))
prove(x * (y + z) == x * y + x * z)
prove(x ** 2 >= 0)

# вещественные
x = FP("x", Float64())
y = FP("y", Float64())
z = FP("z", Float64())

# из-за особенностей реализации вещественных чисел в Z3 их необходимо приводить к рациональным. В типе FP есть значения NaN, +0.0, -0.0, что в реальной математике отсутствует.
prove(fpToReal(x) + fpToReal(y) == fpToReal(y) + fpToReal(x)) # коммутативность
prove(((fpToReal(x) + fpToReal(y)) + fpToReal(z)) == ((fpToReal(x) + (fpToReal(y) + fpToReal(z))))) # ассоциативность
prove(fpToReal(x) + 0 == fpToReal(x)) # существование "0"
prove(1 * x == x) # существование "1"
prove(Or(fpToReal(x) > 0, fpToReal(x) < 0, fpToReal(x) == 0))
prove(fpToReal(x) * (fpToReal(y) + fpToReal(z)) == fpToReal(x) * fpToReal(y) + fpToReal(x) * fpToReal(z))
prove(fpToReal(x) ** 2 >= 0)

# битовые
x = BitVec("x", 64)
y = BitVec("y", 64)
z = BitVec("z", 64)

prove(x + y == y + x) # коммутативность
prove(((x + y) + z) == ((x + (y + z)))) # ассоциативность
prove(x + 0 == x) # существование "0"
prove(1 * x == x) # существование "1"
prove(Or(x > 0, x < 0, x == 0))
prove(x * (y + z) == x * y + x * z)
# prove(x ** 2 >= 0)  # TypeError, в Z3 битвекторы не поддерживают оператор возведения в степень.
prove(x * x >= 0)  # для битовых векторов это правило не может быть доказано, поскольку можно найти значение, которое приведет к модульному переполнению и сменится старший бит, что приведет к смене знака
prove(Implies(
    And(x >= -2**20, x <= 2**20),
    x*x >= 0
))  # можно ограничить значения х -2 ** 31 и 2 ** 31 (в примере я оставил 2 ** 20 - доказательство этой степени уже потребовало 7+ сек) - тогда переполнения не будет.

## Придумать задачи на доказательство теорем

### Решение системы уравнений
Система уравнений x+y=5, x−y=1 имеет ровно одно решение

In [ ]:
x1, y1 = Ints("x1 y1")

existence = Exists([x1, y1], And(x1 + y1 == 5, x1 - y1 == 1))
prove(existence)


x2, y2 = Ints("x2 y2")

def is_solution(x, y):
    return And(
        x + y == 5,
        x - y == 1
    )

not_unique = Exists(
    [x1, y1, x2, y2],
    And(
        is_solution(x1, y1),
        is_solution(x2, y2),
        Or(x1 != x2, y1 != y2)
    )
)

prove(Not(not_unique))


### Свойства степеней
Докажем, что 
- x ** 1 = x
- x **(n + m) = (x ** n) * (x ** m)
- (x ** n) ** m = x ** (n * m)

In [ ]:
# Функция возведения в степень
power = Function("power", IntSort(), IntSort(), IntSort())
x, y, z = Ints("x y z")
n, m, k = Ints("n m k")

# Аксиомы
power_axioms = [
    ForAll([x], power(x, 0) == 1),
    ForAll([x, n], Implies(n > 0, power(x, n) == x * power(x, n - 1))),
]

# x ** 1 = x
theorem = ForAll([x], power(x, 1) == x)
prove(Implies(And(power_axioms), theorem))

# x **(n + m) = (x ** n) * (x ** m)
theorem = ForAll([x, n, m], 
    Implies(And(n >= 0, n < 5, m >= 0, m < 5), 
            power(x, n + m) == power(x, n) * power(x, m)))
prove(Implies(And(power_axioms), theorem))

# (x ** n) ** m = x ** (n * m)
theorem = ForAll([x, n, m],
    Implies(And(n >= 0, n < 4, m >= 0, m < 4),
            power(power(x, n), m) == power(x, n * m)))
prove(Implies(And(power_axioms), theorem))

# Ограничения верхних границ связаны с временем поиска контр-примеров. На этих границах мой комп еще мог в разумное время доказать теоремы.

### Факториал
n! > 2 ** n при n >= 4

In [ ]:
n = Int('n')
factorial = Function("factorial", IntSort(), IntSort())

# factorial definition
factorial_def = [
    factorial(0) == 1,
    ForAll([n], Implies(n > 0, factorial(n) == n * factorial(n - 1)))
]

theorem = ForAll([n], Implies(And(n >= 4, n < 30), factorial(n) > 2 ** n))
prove(Implies(And(factorial_def), theorem))

# Ограничения верхних границ связаны с временем поиска контр-примеров. На этих границах мой комп еще мог в разумное время доказать теоремы.

### Сравнение среднего арифметического и среднего геометрического для двух чисел
Среднее арифметическое >= среднего геометрического (для 2 чисел)

In [ ]:
x, y = Reals('x y')

arifm_mean = Function("arifm_mean", RealSort(), RealSort(), RealSort())

geometric_mean = Function("geometric_mean", RealSort(), RealSort(), RealSort())

# definitions
arif_def = [
    ForAll([x, y], Implies(And(x > 0, y > 0), arifm_mean(x, y) == (x + y) / 2))
]
geom_def = [
    ForAll([x, y], Implies(And(x > 0, y > 0), geometric_mean(x, y) == Sqrt(x * y)))
]

# theorem
theorem = ForAll([x, y], Implies(And(x > 0, y > 0), arifm_mean(x, y) >= geometric_mean(x, y)))

prove(Implies(And(arif_def + geom_def), theorem))